In [101]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download
import json

dense_embs = load_dataset("doruktarhan6/flickr30k-blip-dense", data_files={"img_emb": "img_embs.parquet", "text_emb": "text_embs.parquet"}, keep_in_memory=True).with_format("numpy")
meta_data = json.load(open(hf_hub_download(
    repo_id="doruktarhan6/flickr30k-blip-dense", repo_type="dataset", filename="dataset_meta.json")))

In [147]:
from datasets import load_dataset
from huggingface_hub import hf_hub_download
import json

dense_embs_lsr = load_dataset("lsr42/flickr30k-blip-dense", data_files={"img_emb": "img_embs.parquet", "text_emb": "text_embs.parquet"}, keep_in_memory=True).with_format("numpy")
meta_data_lsr = json.load(open(hf_hub_download(
    repo_id="lsr42/flickr30k-blip-dense", repo_type="dataset", filename="dataset_meta.json")))

In [102]:
dense_embs.keys()

dict_keys(['img_emb', 'text_emb'])

In [131]:
meta_data.keys()

dict_keys(['images', 'dataset'])

In [130]:
dense_embs['img_emb'][0].keys()

dict_keys(['id', 'emb'])

In [135]:
meta_data.keys(), meta_data['dataset'], meta_data['images'][0].keys()

(dict_keys(['images', 'dataset']),
 'flickr30k',
 dict_keys(['sentids', 'imgid', 'sentences', 'split', 'filename']))

In [149]:
type(meta_data['images'][0]['imgid']), type(dense_embs['img_emb'][0]['id']), type(dense_embs_lsr['img_emb'][0]['id']),type(meta_data_lsr['images'][0]['imgid'])

(int, numpy.int64, numpy.str_, int)

In [ ]:
import pandas as pd
metadata_df = pd.DataFrame(meta_data['images'])
metadata_df[120:130]



,sentids,imgid,sentences,split,filename
120,"[600, 601, 602, 603, 604]",120,"[{'tokens': ['a', 'man', 'in', 'a', 'black', '...",train,103625356.jpg
121,"[605, 606, 607, 608, 609]",121,"[{'tokens': ['a', 'caucasian', 'young', 'woman...",train,103631543.jpg
122,"[610, 611, 612, 613, 614]",122,"[{'tokens': ['a', 'woman', 'in', 'a', 'blue', ...",train,1039360778.jpg
123,"[615, 616, 617, 618, 619]",123,"[{'tokens': ['someone', 'is', 'standing', 'on'...",train,103953336.jpg
124,"[620, 621, 622, 623, 624]",124,"[{'tokens': ['a', 'bride', 'in', 'a', 'light',...",test,1039637574.jpg
125,"[625, 626, 627, 628, 629]",125,"[{'tokens': ['young', 'white', 'boy', 'on', 'a...",train,10404007.jpg
126,"[630, 631, 632, 633, 634]",126,"[{'tokens': ['a', 'man', 'is', 'cooking', 'wha...",train,1040426962.jpg
127,"[635, 636, 637, 638, 639]",127,"[{'tokens': ['three', 'people', 'are', 'on', '...",train,104136873.jpg
128,"[640, 641, 642, 643, 644]",128,"[{'tokens': ['a', 'group', 'of', 'tourists', '...",train,104180524.jpg
129,"[645, 646, 647, 648, 649]",129,"[{'tokens': ['a', 'young', 'boy', 'wearing', '...",train,1042020065.jpg


In [137]:
# Convert dense_embs['img_emb']['id'] to a set for faster lookup
dense_img_ids = set(int(item['id']) for item in dense_embs['img_emb'])

# Filter images in metadata
filtered_images = [
    image for image in meta_data['images']
    if image['imgid'] in dense_img_ids
]

# Create the new metadata dictionary with filtered images
filtered_metadata = {
    "dataset": meta_data['dataset'],
    "images": filtered_images
}

filtered_metadata['images'][0]


{'sentids': [0, 1, 2, 3, 4],
 'imgid': 0,
 'sentences': [{'tokens': ['two',
    'young',
    'guys',
    'with',
    'shaggy',
    'hair',
    'look',
    'at',
    'their',
    'hands',
    'while',
    'hanging',
    'out',
    'in',
    'the',
    'yard'],
   'raw': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.',
   'imgid': 0,
   'sentid': 0},
  {'tokens': ['two',
    'young',
    'white',
    'males',
    'are',
    'outside',
    'near',
    'many',
    'bushes'],
   'raw': 'Two young, White males are outside near many bushes.',
   'imgid': 0,
   'sentid': 1},
  {'tokens': ['two',
    'men',
    'in',
    'green',
    'shirts',
    'are',
    'standing',
    'in',
    'a',
    'yard'],
   'raw': 'Two men in green shirts are standing in a yard.',
   'imgid': 0,
   'sentid': 2},
  {'tokens': ['a',
    'man',
    'in',
    'a',
    'blue',
    'shirt',
    'standing',
    'in',
    'a',
    'garden'],
   'raw': 'A man in a blue shirt standing in

In [144]:
len(filtered_metadata['images']), len(meta_data['images'])

(30402, 31014)

In [139]:
# Save filtered metadata back to JSON
filtered_metadata_path = "dataset_meta.json"
with open(filtered_metadata_path, "w") as file:
    json.dump(filtered_metadata, file, indent=2)

print(f"Filtered metadata saved to {filtered_metadata_path}")


Filtered metadata saved to dataset_meta.json


In [143]:
pd.DataFrame(filtered_metadata['images'])[120:130]

,sentids,imgid,sentences,split,filename
120,"[600, 601, 602, 603, 604]",120,"[{'tokens': ['a', 'man', 'in', 'a', 'black', '...",train,103625356.jpg
121,"[605, 606, 607, 608, 609]",121,"[{'tokens': ['a', 'caucasian', 'young', 'woman...",train,103631543.jpg
122,"[610, 611, 612, 613, 614]",122,"[{'tokens': ['a', 'woman', 'in', 'a', 'blue', ...",train,1039360778.jpg
123,"[615, 616, 617, 618, 619]",123,"[{'tokens': ['someone', 'is', 'standing', 'on'...",train,103953336.jpg
124,"[620, 621, 622, 623, 624]",124,"[{'tokens': ['a', 'bride', 'in', 'a', 'light',...",test,1039637574.jpg
125,"[625, 626, 627, 628, 629]",125,"[{'tokens': ['young', 'white', 'boy', 'on', 'a...",train,10404007.jpg
126,"[630, 631, 632, 633, 634]",126,"[{'tokens': ['a', 'man', 'is', 'cooking', 'wha...",train,1040426962.jpg
127,"[635, 636, 637, 638, 639]",127,"[{'tokens': ['three', 'people', 'are', 'on', '...",train,104136873.jpg
128,"[640, 641, 642, 643, 644]",128,"[{'tokens': ['a', 'group', 'of', 'tourists', '...",train,104180524.jpg
129,"[650, 651, 652, 653, 654]",130,"[{'tokens': ['a', 'older', 'man', 'holds', 'an...",train,1042359076.jpg


In [146]:
dense_embs['img_emb'][:]['id'][120:130]

array([120, 121, 122, 123, 124, 125, 126, 127, 128, 130])

In [92]:
import pandas as pd
metadata_df = pd.DataFrame(meta_data['images'])


In [100]:
exists_in_meta = any(image['id'] == 125 for image in dense_embs['img_emb'])
print(f"ID '125' exists in metadata: {exists_in_meta}")


ID '125' exists in metadata: False


In [88]:
dense_embs['img_emb'].to_pandas().head()

,id,emb
0,0,"[-0.022006208077073097, 0.09969383478164673, 0..."
1,1,"[0.09531231224536896, 0.04221922904253006, -0...."
2,2,"[-0.039520345628261566, 0.10081849247217178, 0..."
3,3,"[0.09637868404388428, 0.12081631273031235, -0...."
4,4,"[0.016611870378255844, 0.10799410194158554, -0..."


In [15]:
dense_embs['img_emb']['emb'].shape

(31014, 256)

In [16]:
dense_embs['text_emb']['emb'].shape

(155070, 256)

In [17]:
dense_embs['img_emb']

Dataset({
    features: ['id', 'emb'],
    num_rows: 31014
})

In [5]:
meta_data.keys()

dict_keys(['images', 'dataset'])

In [6]:
meta_data['dataset']


'coco'

In [7]:
meta_data['images'][0].keys()

dict_keys(['filepath', 'sentids', 'filename', 'imgid', 'split', 'sentences', 'cocoid'])

In [40]:
meta_data['images'][1]

{'filepath': 'val2014',
 'sentids': [681330, 686718, 688839, 693159, 693204],
 'filename': 'COCO_val2014_000000522418.jpg',
 'imgid': 1,
 'split': 'restval',
 'sentences': [{'tokens': ['a',
    'woman',
    'wearing',
    'a',
    'net',
    'on',
    'her',
    'head',
    'cutting',
    'a',
    'cake'],
   'raw': 'A woman wearing a net on her head cutting a cake. ',
   'imgid': 1,
   'sentid': 681330},
  {'tokens': ['a', 'woman', 'cutting', 'a', 'large', 'white', 'sheet', 'cake'],
   'raw': 'A woman cutting a large white sheet cake.',
   'imgid': 1,
   'sentid': 686718},
  {'tokens': ['a',
    'woman',
    'wearing',
    'a',
    'hair',
    'net',
    'cutting',
    'a',
    'large',
    'sheet',
    'cake'],
   'raw': 'A woman wearing a hair net cutting a large sheet cake.',
   'imgid': 1,
   'sentid': 688839},
  {'tokens': ['there',
    'is',
    'a',
    'woman',
    'that',
    'is',
    'cutting',
    'a',
    'white',
    'cake'],
   'raw': 'there is a woman that is cutting a

In [ ]:
# Preprocess text embeddings for fast lookup, normalizing keys to strings
text_emb_lookup = {str(item['id']): item['emb'] for item in dense_embs['text_emb']}  # Ensure all keys are strings




Image embedding shape: (256,)
Number of text embeddings found: 5
Text embedding shapes: [(256,), (256,), (256,), (256,), (256,)]


In [48]:
# Retrieve image embedding
imgid = meta_data['images'][0]['imgid']
example_img_embed = dense_embs['img_emb'][imgid]['emb']

# Retrieve text embeddings for the corresponding sentence IDs
sentids = meta_data['images'][0]['sentids']
text_embeddings = [text_emb_lookup[str(sentid)] for sentid in sentids if str(sentid) in text_emb_lookup]

# Output
print(f"Image embedding shape: {example_img_embed.shape}")
print(f"Number of text embeddings found: {len(text_embeddings)}")
print(f"Text embedding shapes: {[emb.shape for emb in text_embeddings]}")

Image embedding shape: (256,)
Number of text embeddings found: 5
Text embedding shapes: [(256,), (256,), (256,), (256,), (256,)]


In [49]:

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
"""
for i in range(len(text_embeddings)):
    cosine_sim_matrix = cosine_similarity(example_img_embed.reshape(-1,1), text_embeddings[i].reshape(-1,1))
    print(f"Text embedding {i} cosine similarity: {cosine_sim_matrix}")

"""
for i in range(len(text_embeddings)):
    pairwise_cosine_sim = np.matmul(example_img_embed, text_embeddings[i].T)
    print(f"Text embedding {i} cosine similarity: {pairwise_cosine_sim}")





Text embedding 0 cosine similarity: 0.43064552545547485
Text embedding 1 cosine similarity: 0.42722970247268677
Text embedding 2 cosine similarity: 0.33973366022109985
Text embedding 3 cosine similarity: 0.4240642786026001
Text embedding 4 cosine similarity: 0.4385703504085541


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import json

# Define paths
path_to_dataset_folder = '/Users/doruktarhan/Desktop/MSCOCO_Dataset' #dataset images folder path
# Use glob to find all image files across subdirectories
image_files = glob(f'{path_to_dataset_folder}/**/*.jpg', recursive=True)
all_image_paths = {os.path.basename(file): file for file in image_files}




# Load metadata
meta_data_path = 'data/dataset_coco.json' #metadata file path
with open(meta_data_path, 'r') as f:
    meta_data_kaggle = json.load(f)

# Display images and metadata for the first 5 instances
for idx, metadata_item in enumerate(meta_data_kaggle['images'][:5]):
    filename = metadata_item["filename"]  
    captions = [sentence["raw"] for sentence in metadata_item["sentences"]]  # Captions list
    
    # Use the glob-based lookup dictionary to find the image path
    image_path = all_image_paths.get(filename)
    if not image_path:
        print(f"Image not found: {filename}")
        continue

    # Load the image
    image = Image.open(image_path)

    # Display the metadata and image
    print(f"Metadata for instance {idx + 1}:")
    print(f"Filename: {filename}")
    print(f"Captions: {captions}")

    # Plot the image and captions
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Captions: {captions[0]}")
    plt.show()


In [ ]:
meta_data_kaggle['images'][0].keys()

In [ ]:
import pandas as pd

# Convert metadata to a pandas DataFrame
metadata_images = meta_data_kaggle['images']  # Extract the 'images' key
df = pd.DataFrame(metadata_images)

# Display the first few rows of the DataFrame
print(df.head())

# Check unique splits and their counts
split_counts = df['split'].value_counts()

print("\nUnique Splits:")
print(split_counts)


### BLIP Encoder Representations

In [ ]:
from transformers import BlipProcessor, BlipForImageTextRetrieval
from PIL import Image
import torch
import os
import json


# Load the BLIP model and processor
model_name = "Salesforce/blip-itm-large-coco" #model card for image text matching
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForImageTextRetrieval.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


#load metadata
meta_data_path = 'data/dataset_coco.json'
with open(meta_data_path, 'r') as f:
    meta_data_kaggle = json.load(f)


#image path 
path_to_dataset_folder = '/Users/doruktarhan/Desktop/MSCOCO_Dataset' #dataset images folder path




In [ ]:
from PIL import Image
import torch

# Define paths
path_to_dataset_folder = '/Users/doruktarhan/Desktop/MSCOCO_Dataset' #dataset images folder path
# Use glob to find all image files across subdirectories
image_files = glob(f'{path_to_dataset_folder}/**/*.jpg', recursive=True)
all_image_paths = {os.path.basename(file): file for file in image_files}




# Load metadata
meta_data_path = 'data/dataset_coco.json' #metadata file path
with open(meta_data_path, 'r') as f:
    meta_data_kaggle = json.load(f)

# Display images and metadata for the first 5 instances
for idx, metadata_item in enumerate(meta_data_kaggle['images'][:1]):
    filename = metadata_item["filename"]  # e.g., '1000092795.jpg'
    captions = [sentence["raw"] for sentence in metadata_item["sentences"]]  # Captions list
    
    # Use the glob-based lookup dictionary to find the image path
    image_path = all_image_paths.get(filename)
    if not image_path:
        print(f"Image not found: {filename}")
        continue

    # Load the image
    image = Image.open(image_path)

    # Display the metadata and image
    print(f"Metadata for instance {idx + 1}:")
    print(f"Filename: {filename}")
    print(f"Captions: {captions}")

    # Plot the image and captions
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Captions: {captions[0]}")
    plt.show()



In [ ]:
from torch.nn.functional import softmax

for caption in captions:
    inputs = processor(images=image, text=caption, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)
    print(f'Caption: {caption}')
    print(outputs['itm_score'])
    probabilities = softmax(outputs['itm_score'], dim=1)
    print(probabilities)
    print(outputs['question_embeds'].shape)
    print("\n")



In [ ]:
outputs.keys()

In [ ]:
caption = 'Man riding on a motorcycle with blue helmet'
inputs = processor(images=image, text=caption, return_tensors="pt")

# Perform inference
outputs = model(**inputs)
print(f'Caption: {caption}')
print(f"Output tensor itm matching score: {outputs['itm_score']}")
probabilities = softmax(outputs['itm_score'], dim=1)
print(f"Probabilities after softmax: {probabilities}")

print(f"Question embedding shape: {outputs['question_embeds'].shape}")
print(f"Image embedding shape: {outputs['last_hidden_state'].shape}")

print("\n")

In [ ]:
from torch.nn.functional import normalize

proj_text_embedding = normalize(model.text_proj(outputs.question_embeds[:,0,:]))
proj_image_embedding = normalize(model.vision_proj(outputs.last_hidden_state[:,0,:]))


print(f"Image projection shape: {proj_image_embedding.shape}")
print(f"Text projection shape: {proj_text_embedding.shape}")

# Compute pairwise cosine similarity (matrix)
pairwise_cosine_sim = torch.matmul(proj_image_embedding, proj_text_embedding.T)

print(f"Pairwise cosine similarity:\n{pairwise_cosine_sim}")

In [3]:
import torch

# Load the saved embeddings
file_path = "test_embeddings/blip_test/test_batch.pt"
embeddings = torch.load(file_path)

# Extract the embeddings
image_embeds = embeddings["image_embeds"]  # (batch_size, embedding_dim)
text_embeds = embeddings["text_embeds"]    # (batch_size, embedding_dim)

# Normalize the embeddings
image_embeds = torch.nn.functional.normalize(image_embeds, p=2, dim=1)  # Normalize along embedding dimension
text_embeds = torch.nn.functional.normalize(text_embeds, p=2, dim=1)

# Compute pairwise cosine similarity for each image-text pair
cosine_similarities = torch.sum(image_embeds * text_embeds, dim=1)

# Print the results
print("Cosine similarities for each image-text pair:")
for i, sim in enumerate(cosine_similarities):
    print(f"Pair {i+1}: {sim.item()}")


Cosine similarities for each image-text pair:
Pair 1: 0.4717683494091034
Pair 2: 0.4680973291397095
Pair 3: 0.4207531213760376
Pair 4: 0.5063284635543823
Pair 5: 0.4703652560710907
Pair 6: 0.4047545790672302
Pair 7: 0.46468424797058105
Pair 8: 0.46485379338264465
Pair 9: 0.4473761022090912
Pair 10: 0.5034165978431702
Pair 11: 0.42352989315986633
Pair 12: 0.5207198262214661
Pair 13: 0.4432687759399414
Pair 14: 0.44502633810043335
Pair 15: 0.5233748555183411
Pair 16: 0.44414687156677246


/var/folders/p9/056tygps4hv_9gn5wdt55xwh0000gn/T/ipykernel_78812/1152185754.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(file_path)


In [6]:
import pandas as pd
import numpy as np
import torch

# Paths to the Parquet files
img_embs_path = "embeddings_small/img_embs.parquet"
text_embs_path = "embeddings_small/text_embs.parquet"

# Load Parquet files
img_embs_df = pd.read_parquet(img_embs_path)
text_embs_df = pd.read_parquet(text_embs_path)

# Ensure one-to-one correspondence
assert len(img_embs_df) == len(text_embs_df), "Mismatch in image and text embeddings!"

# Extract embeddings and convert them to tensors
img_embs = torch.tensor(np.vstack(img_embs_df['embedding'].values))
text_embs = torch.tensor(np.vstack(text_embs_df['embedding'].values))

# Normalize embeddings for cosine similarity (optional, depends on the model output)
img_embs_normalized = torch.nn.functional.normalize(img_embs, dim=1)
text_embs_normalized = torch.nn.functional.normalize(text_embs, dim=1)

# Compute pairwise cosine similarity using matrix multiplication
cosine_similarities = torch.matmul(img_embs_normalized, text_embs_normalized.T)

# Extract diagonal elements for one-to-one correspondence cosine similarities
one_to_one_cosine_similarities = cosine_similarities.diag()

# Print the cosine similarities
for idx, sim in enumerate(one_to_one_cosine_similarities):
    print(f"Image {idx} and Text {idx} Cosine Similarity: {sim.item()}")


Image 0 and Text 0 Cosine Similarity: 0.4687182905649778
Image 1 and Text 1 Cosine Similarity: 0.4822275800169071
Image 2 and Text 2 Cosine Similarity: 0.44469575617662155
Image 3 and Text 3 Cosine Similarity: 0.4633115597567263
Image 4 and Text 4 Cosine Similarity: 0.4925434587492448
Image 5 and Text 5 Cosine Similarity: 0.5880840729854441
Image 6 and Text 6 Cosine Similarity: 0.5194537997895264
Image 7 and Text 7 Cosine Similarity: 0.46692864103672294
Image 8 and Text 8 Cosine Similarity: 0.5903741106656454
Image 9 and Text 9 Cosine Similarity: 0.4781363231533651
Image 10 and Text 10 Cosine Similarity: 0.4350797959041108
Image 11 and Text 11 Cosine Similarity: 0.43959263238461244
Image 12 and Text 12 Cosine Similarity: 0.43109933663207517
Image 13 and Text 13 Cosine Similarity: 0.45771729806122613
Image 14 and Text 14 Cosine Similarity: 0.44786657768916066
Image 15 and Text 15 Cosine Similarity: 0.46102534364631664
Image 16 and Text 16 Cosine Similarity: 0.4925554589395006
Image 17 a

In [9]:
len(img_embs_df['index'].unique())

32

In [31]:
import json

# Load metadata
meta_data_path = 'meta_data/dataset_coco.json'  # Replace with your metadata path
image_name = 'COCO_val2014_000000017778.jpg'

with open(meta_data_path, 'r') as file:
    metadata = json.load(file)

# Find the image
for image_data in metadata['images']:
    if image_data['filename'] == image_name:
        print(image_data)
        break
else:
    print(f"Image '{image_name}' not found in metadata.")


{'filepath': 'val2014', 'sentids': [105783, 106152, 111315, 116598, 122208], 'filename': 'COCO_val2014_000000017778.jpg', 'imgid': 19779, 'split': 'restval', 'sentences': [{'tokens': ['a', 'hummingbird', 'flying', 'above', 'a', 'bunch', 'of', 'small', 'red', 'and', 'white', 'flowers'], 'raw': 'a hummingbird flying above a bunch of small red and white flowers', 'imgid': 19779, 'sentid': 105783}, {'tokens': ['the', 'gerania', 'are', 'flowering', 'with', 'pink', 'white', 'and', 'red', 'florets'], 'raw': 'The gerania are flowering with pink, white, and red florets.', 'imgid': 19779, 'sentid': 106152}, {'tokens': ['a', 'humming', 'bird', 'above', 'a', 'fushia', 'flower', 'among', 'green', 'plants'], 'raw': 'A humming bird above a fushia flower among green plants. ', 'imgid': 19779, 'sentid': 111315}, {'tokens': ['an', 'insect', 'perches', 'on', 'a', 'flower', 'amidst', 'green', 'leaves'], 'raw': 'An insect perches on a flower amidst green leaves.', 'imgid': 19779, 'sentid': 116598}, {'token

In [33]:
from PIL import Image

# Load the image
image_path = ('/Users/doruktarhan/Desktop/MSCOCO_trial_images_small/val2014/COCO_val2014_000000017953.jpg')
image = Image.open(image_path)


In [46]:
import os
directory = "embeddings_small/"

dense_embs = pd.read_parquet(os.path.join(directory, "img_embs.parquet"))


In [47]:
len(dense_embs)

64

In [81]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet("/Users/doruktarhan/Desktop/LSR_Encoder_Results/flickr30k-blip-dense_old/text_embs.parquet")



# Save it back without the index
df.to_parquet("/Users/doruktarhan/Desktop/LSR_Encoder_Results/flickr30k-blip-dense/text_embs.parquet", index=False)


In [82]:
df.head()

,id,emb
0,0,"[-0.0025575943291187286, 0.0009181360946968198..."
1,1,"[-0.07348638027906418, 0.038891274482011795, 0..."
2,2,"[0.03266528621315956, 0.11210073530673981, -0...."
3,3,"[0.013137374073266983, -0.000507828954141587, ..."
4,4,"[0.05367390066385269, -0.05757979676127434, -0..."


In [83]:
df =  pd.read_parquet("/Users/doruktarhan/Desktop/LSR_Encoder_Results/flickr30k-blip-dense/img_embs.parquet")
df.head()

,id,emb
0,0,"[-0.022006208077073097, 0.09969383478164673, 0..."
1,1,"[0.09531231224536896, 0.04221922904253006, -0...."
2,2,"[-0.039520345628261566, 0.10081849247217178, 0..."
3,3,"[0.09637868404388428, 0.12081631273031235, -0...."
4,4,"[0.016611870378255844, 0.10799410194158554, -0..."


### Truncated image error

In [150]:
from PIL import Image
import os

def find_and_remove_corrupted_images(image_paths):
    """
    Identify and delete corrupted images.
    
    Args:
        image_paths (list): List of image file paths.
    """
    corrupted_images = []
    for img_path in image_paths:
        try:
            with Image.open(img_path) as img:
                img.verify()  # Check if the file is an image
        except (OSError, FileNotFoundError) as e:
            print(f"Corrupted image: {img_path}, Error: {e}")
            corrupted_images.append(img_path)

    # Remove corrupted images
    for img_path in corrupted_images:
        try:
            os.remove(img_path)
            print(f"Removed: {img_path}")
        except Exception as e:
            print(f"Error deleting file {img_path}: {e}")

# Example usage
image_folder = "/Users/doruktarhan/Desktop/MSCOCO_Dataset/train2014"
all_images = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
find_and_remove_corrupted_images(all_images)


Corrupted image: /Users/doruktarhan/Desktop/MSCOCO_Dataset/train2014/.DS_Store, Error: cannot identify image file '/Users/doruktarhan/Desktop/MSCOCO_Dataset/train2014/.DS_Store'
Removed: /Users/doruktarhan/Desktop/MSCOCO_Dataset/train2014/.DS_Store
